In [ ]:
%pylab inline
import imageio
import skan
import networkx as nx
import glob
from tqdm.notebook import tqdm
import scipy.ndimage as ndi
from skimage import morphology

In [ ]:
recondir = "/media/maxc/Data/Tomography/Time_Series/PZ103/Skeletons/Primary/" #directory where skeletons live
flist = sorted(glob.glob(recondir + "*.tif"))
#All skeletons in 4D array. If they are too lrg for memory loading needs to be in loop
skimgs = array([imageio.volread(i) for i in flist])

In [ ]:
zoff = int(skimgs[0].shape[0]*0.01) # How many slices behind previous Z_max to check
kern = zeros((3,3,3), dtype=bool) # Kernel for dilation
kern[1] = 1 # Dilate only in X-Y, i.e. if a pixel is True, surround by True.

nbranches = zeros(len(flist)) # Number of branches for a particular time slice
zmax = zeros(len(flist)) # Z_max for time slices


zmax[0] = where(any(skimgs[0], axis=(1,2)))[0].max() # Initial Z_max
skel = skan.Skeleton(skimgs[0]) # Skeleton graph from binary skeleton volume
G = nx.from_scipy_sparse_matrix(skel.graph) # Turn sparse connectivity matrix into networkx graph
ncons = array([len(G.adj[k]) for k in arange(len(G.adj))]) # How many edge are connected to each node
nbranches[0] = sum(ncons>=3) # Branch nodes are connected to 3 edges or greater

In [ ]:
for i, fname in enumerate(tqdm(flist[1:])):
    widx = int(zmax[i]-zoff) if zmax[i] - zoff >= 0 else 0 # Look at slices >= previous Z_max - zoff
    ps = skimgs[i,widx:] # Previous skeleton
    ps = ndi.binary_dilation(ps, kern) # Dilate previous skeleton in X-Y
    ws = skimgs[i+1,widx:] # Current working skeleton
    sd = (ws^ps)&ws # Subtract previous skeleton from current. Returns true if voxel is only in ws
    if sum(sd) <= 510: # Check that subtraction isn't all False or too few True to construct graph
        nbranches[i+1] = 0 # Num branches is 0 if all False or too few True (510 value is two voxels)
        zmax[i+1] = zmax[i]
        continue
    skel = skan.Skeleton(sd) # Skeleton graph from subtracted binary skeletons
    G = nx.from_scipy_sparse_matrix(skel.graph)
    ncons = array([len(G.adj[k]) for k in arange(len(G.adj))])
    nbranches[i+1] = sum(ncons>=3)
    zmax[i+1] = where(any(skimgs[i+1], axis=(1,2)))[0].max() # Z_max of current working skeleton

In [ ]:
import pandas
from os.path import basename
flist = [basename(i) for i in flist]
df = pandas.DataFrame({"Scan":flist, "NumBranches":nbranches})
plot(nbranches)
show()
df.to_csv("PZ103_deltabranch.csv")